In [ ]:
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)
warnings.filterwarnings('ignore',category=DeprecationWarning)

# Import relevant libraries
import numpy as np
from numpy import asarray
from PIL import Image
import os
import random
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils.vis_utils import plot_model
from keras.losses import BinaryCrossentropy
import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from matplotlib import pyplot
from matplotlib.patches import Rectangle
from mtcnn.mtcnn import MTCNN

%matplotlib inline

# Data Parsing
## All the user has to do is to insert the folder with the faces images and the folder without faces.
### Then, we automatically divide the data into training (about 75%) and testing (about 25%), and format the images so they're a 256x256 image (The user can change the value for example: 512x512 or 1024x1024) but the model will take more time to train.

In [ ]:
# Source
# https://note.nkmk.me/en/python-pillow-square-circle-thumbnail/
def expand2square(pil_img, background_color):
    width, height = pil_img.size
    if width == height:
        return pil_img
    elif width > height:
        result = Image.new(pil_img.mode, (width, width), background_color)
        result.paste(pil_img, (0, (width - height) // 2))
        return result
    else:
        result = Image.new(pil_img.mode, (height, height), background_color)
        result.paste(pil_img, ((height - width) // 2, 0))
        return result

def square_img(directory, size=256):
    training_dir = directory + '_train'
    testing_dir = directory + '_test'
    
    # Clean both test and training folders, and recreate them
    if os.path.isdir(training_dir):
        for f in os.listdir(training_dir):
            os.remove(training_dir + '/' + f)
        os.rmdir(training_dir)
    if os.path.isdir(testing_dir):
        for f in os.listdir(testing_dir):
            os.remove(testing_dir + '/' + f)
        os.rmdir(testing_dir)
    os.mkdir(training_dir)
    os.mkdir(testing_dir)
    
    # 25% of images for test and 75% for training
    path, dirs, files = next(os.walk(directory))
    count_examples = 0
    test_examples = int(len(files) * 0.25)
    print("Getting test data")
    
    # Save all in training dir
    for filename in os.listdir(directory):
        f = directory + '/' + filename
        im = Image.open(f)
        new_image = expand2square(im, (0, 0, 0)).resize((size, size), Image.LANCZOS)
        newfilename = training_dir + '/' + filename
        new_image.save(newfilename)
    # Pass from training folder to testing
    while True:
        for filename in os.listdir(training_dir):
            f = training_dir + '/' + filename
            im = Image.open(f)
            # Add black bars to the side of the images
            new_image = expand2square(im, (0, 0, 0)).resize((size, size), Image.LANCZOS)
            # Randomly pass some images to test folder
            test_p = random.randint(0, 12)
            if test_p == 6:
                newfilename = testing_dir + '/' + filename
                new_image.save(newfilename)
                os.remove(training_dir + '/' + filename)
                count_examples += 1
            if count_examples == test_examples:
                break
        if count_examples == test_examples:
                break
            
    return training_dir, testing_dir

In [ ]:
# Insert the name of the directory
faces_directory = 'faces'
no_faces_directory = 'no_faces'

# Square the image and turn it into a 256x256 jpeg (optional: pass the size as second argument ex: 512)
faces_dir_train, faces_dir_test = square_img(faces_directory)
no_faces_dir_train, no_faces_dir_test = square_img(no_faces_directory)

# Training Images
### Now we parse the training images and label them as 0 (no face) or 1 (face)


In [ ]:
ls = []
label_ls = []
i = 0 
j = 0
interval = 1
path, dirs, files = next(os.walk(faces_dir_train))
print("Total images: ", len(files))
if int(len(files) / 1000) > 0:
    interval = int(len(files) / 1000)
for filename in os.listdir(faces_dir_train):
    i += 1
    if (i % interval == 0):
        j += 1
        img = Image.open(faces_dir_train + '/' + filename)
        data = asarray(img)
        ls.append(data)
        label_ls.append(1)
print("Number of images used to train the model: ", j)

i = 0 
j = 0
interval = 1
path, dirs, files = next(os.walk(no_faces_dir_train))
print("Total images: ", len(files))
if int(len(files) / 1000) > 0:
    interval = int(len(files) / 1000)
for filename in os.listdir(no_faces_dir_train):
    i += 1
    if (i % interval == 0):
        j += 1
        img = Image.open(no_faces_dir_train + '/' + filename)
        data = asarray(img)
        ls.append(data)
        label_ls.append(0)
print("Number of images used to train the model: ", j)


imgs_train = np.array(ls, dtype='uint8')
label_train = np.array([label_ls])

# Testing images
### Now we parse the testing images and label them as 0 (no face) or 1 (face)

In [ ]:
ls = []
label_ls = []
i = 0 
j = 0
interval = 1
path, dirs, files = next(os.walk(faces_dir_test))
print("Total images: ", len(files))
if int(len(files) / 1000) > 0:
    interval = int(len(files) / 1000)
for filename in os.listdir(faces_dir_test):
    i += 1
    if (i % interval == 0):
        j += 1
        img = Image.open(faces_dir_test + '/' + filename)
        data = asarray(img)
        ls.append(data)
        label_ls.append(1)
print("Number of images used to train the model: ", j)

i = 0 
j = 0
interval = 1
path, dirs, files = next(os.walk(no_faces_dir_test))
print("Total images: ", len(files))
if int(len(files) / 1000) > 0:
    interval = int(len(files) / 1000)
for filename in os.listdir(no_faces_dir_test):
    i += 1
    if (i % interval == 0):
        j += 1
        img = Image.open(no_faces_dir_test + '/' + filename)
        data = asarray(img)
        ls.append(data)
        label_ls.append(0)
print("Number of images used to train the model: ", j)


imgs_test = np.array(ls, dtype='uint8')
label_test = np.array([label_ls])

In [ ]:
# Normalize image vectors
imgs_train_norm = imgs_train / 255.
imgs_test_norm = imgs_test / 255.

# Reshape
label_train_T = label_train.T
label_test_T = label_test.T

print ("Number of training examples = " + str(imgs_train_norm.shape[0]))
print ("Number of testing examples = " + str(imgs_test_norm.shape[0]))
print ("Images train shape: " + str(imgs_train_norm.shape))
print ("Labels train shape: " + str(label_train_T.shape))
print ("Images test shape: " + str(imgs_test_norm.shape))
print ("Labels test shape: " + str(label_test_T.shape))

In [ ]:
def FaceDetectionModel(input_shape):
    """
    Implementation of the FaceDetectionModel.
    
    Arguments:
    input_shape -- shape of the images of the dataset

    Returns:
    model -- a Model() instance in Keras
    """
    # Use the suggested model in the text above to get started, and run through the whole
    # exercise once. Then come back and add more BLOCKS. 
    X_input = Input(input_shape)

    # Zero-Padding: pads the border of X_input with zeroes
    X = ZeroPadding2D((3, 3))(X_input)
    
    # BLOCK 1: CONV -> BN -> RELU -> MAXP
    num_filters = 32
    kernel_size = (7, 7)
    stride = (1, 1)
    X = Conv2D(num_filters, kernel_size, strides = stride, name = 'conv0')(X)

    # BLOCK 2: CONV -> BN -> RELU -> MAXP
    X = BatchNormalization(axis = 3, name = 'bn0')(X)
        
    # BLOCK 3: CONV -> BN -> RELU -> MAXP
    X = Activation('relu')(X)
    
    # BLOCK 4: CONV -> BN -> RELU -> MAXP
    max_pool_size = (2, 2)
    X = MaxPooling2D(max_pool_size, name='max_pool')(X)
       
    # FLATTEN X (means convert it to a vector) + FULLYCONNECTED
    X = Flatten()(X)
    X = Dense(1, activation='sigmoid', name='fc')(X)

    # Create model.
    model = Model(inputs = X_input, outputs = X, name='FaceDetectionModel')
    
    return model

In [ ]:
facedetectionmodel = FaceDetectionModel(imgs_train_norm.shape[1:])

In [ ]:
facedetectionmodel.compile(optimizer='SGD', loss=BinaryCrossentropy(), metrics=["accuracy"])

In [ ]:
facedetectionmodel.summary()

In [ ]:
history = facedetectionmodel.fit(
    imgs_train_norm, label_train_T,
    batch_size = 64,
    epochs = 10,
    validation_data = (imgs_test_norm, label_test_T)
)

In [ ]:
preds = facedetectionmodel.evaluate(x = imgs_test_norm, y = label_test_T)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

In [ ]:
# plots the model in a nice layout and save it as ".png"
plot_model(facedetectionmodel, to_file='FaceDetectionModel.png')

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
def draw_image_with_boxes(filename, result_list):
    data = pyplot.imread(filename)
    pyplot.imshow(data)
    ax = pyplot.gca()
    for result in result_list:
        x, y, width, height = result['box']
        rect = Rectangle((x, y), width, height, fill=False, color='red')
        ax.add_patch(rect)
    pyplot.savefig("result")
    pyplot.show()

In [ ]:
# Test our own images
test_directory = 'test_imgs'
img_path = test_directory + '/person_0179.jpg'
img = image.load_img(img_path, target_size=(256, 256))

x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)


if facedetectionmodel.predict(x)[0][0] == 1:
    face = 0
    print('The image has a face')
    filename = img_path
    pixels = pyplot.imread(filename)
    detector = MTCNN()
    faces = detector.detect_faces(pixels)
    draw_image_with_boxes(filename, faces)

elif facedetectionmodel.predict(x)[0][0] == 0:
    imshow(img)
    face = 1
    print('The image has no faces')

# Calculate accuracy, precision, recall and F1 score

In [ ]:
# Add imgs to list
no_faces_ls = []
faces_ls = []
# Total number of imgs with faces and no faces
tot_faces = 0
tot_no_faces = 0

for filename in os.listdir(test_directory):
    if filename[0] == '0':
        no_faces_ls.append(filename)
        tot_no_faces += 1
    else:
        faces_ls.append(filename)
        tot_faces += 1

# number of true positives, true negatives, false positives and false negatives
num_tp = 0
num_tn = 0
num_fp = 0
num_fn = 0
        
# Testing on all no_faces
print('-------------------')
print('Should be 0 (no faces)')
for img_name in no_faces_ls:
    img_path = test_directory + '/' + img_name
    img = image.load_img(img_path, target_size=(256, 256))

    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    if facedetectionmodel.predict(x)[0][0] == 0:
        num_tn += 1
    else:
        num_fp += 1
    print(img_name + ': ' + str(facedetectionmodel.predict(x)[0][0]))
    
# Testing on all faces
print('-------------------')
print('Should be 1 (faces)')
for img_name in faces_ls:
    img_path = test_directory + '/' + img_name
    img = image.load_img(img_path, target_size=(256, 256))

    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    if facedetectionmodel.predict(x)[0][0] == 0:
        num_fn += 1
    else:
        num_tp += 1
    print(img_name + ': ' + str(facedetectionmodel.predict(x)[0][0]))
    
accuracy = ((num_tp + num_tn) / (num_tp + num_tn + num_fp + num_fn))
precision = num_tp / (num_tp + num_fp)
recall = num_tp / (num_tp + num_fn)
f1_score = 2 * (recall * precision) / (recall + precision)
print('Accuracy: ' + str(accuracy))
print('Precision: ' + str(precision))
print('Recall: ' + str(recall))
print('F1 Score: ' + str(f1_score))